# Task 2: Restaurant Recommendation

**STEP-1 :** Preprocess the dataset by handling missing
values and encoding categorical variables.

In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("C:/Users/AB/Desktop/internship/Dataset.csv")

print("Dataset shape:", df.shape)
print("\nMissing Values Before Processing:\n", df.isnull().sum())

num_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

df['Cuisines'] = df['Cuisines'].str.lower().str.strip()
df['City'] = df['City'].str.lower().str.strip()
df['Rating text'] = df['Rating text'].str.lower().str.strip()

df['Currency'] = df['Currency'].astype(str).str.strip().str.upper()

encode_cols = [col for col in cat_cols if col not in ['Restaurant Name', 'Address', 'Cuisines', 'City', 'Currency']]

label_enc = LabelEncoder()
for col in encode_cols:
    df[col] = label_enc.fit_transform(df[col])

print("\nMissing Values After Processing:\n", df.isnull().sum())

Dataset shape: (9551, 21)

Missing Values Before Processing:
 Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

Missing Values After Processing:
 Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                0
Average Cost for two    0
Currency                0
Has Table booking       0
Has On

**STEP-2 :** Determine the criteria for restaurant
recommendations (e.g., cuisine preference,
price range).

In [14]:
def get_user_preferences(df):
    print(" Please enter your restaurant preferences:\n")

    cuisine = input("\nEnter your preferred cuisine (e.g., Italian, Chinese): ").strip().lower()
    cuisine_list = [c.strip() for c in cuisine.split(',') if c.strip()]

    while True:
        try:
            max_price_range = int(input("Enter your max price range (1 to 4): "))
            if max_price_range in [1, 2, 3, 4]:
                break
            else:
                print("Please enter a number between 1 and 4.")
        except ValueError:
            print("Invalid input. Enter a valid integer between 1 and 4.")

    while True:
        try:
            min_rating = float(input("Enter your minimum acceptable rating (e.g., 3.5): "))
            if 0 <= min_rating <= 5:
                break
            else:
                print("Enter a rating between 0 and 5.")
        except ValueError:
            print("Invalid input. Enter a number like 4.0")

    online_delivery = input("Require online delivery? (yes/no): ").strip().lower()
    while online_delivery not in ['yes', 'no']:
        online_delivery = input("Please enter 'yes' or 'no': ").strip().lower()

    available_currencies = df['Currency'].dropna().unique()
    currency_map = {curr.upper(): curr for curr in available_currencies}  # normalized lookup
    print("\n Available Currencies:")
    print(", ".join(sorted(currency_map.keys())))

    while True:
        currency_input = input("Enter your preferred currency (as shown above): ").strip().upper()
        if currency_input in currency_map:
            currency = currency_map[currency_input]
            break
        else:
            print("Please choose from the listed currency codes.")

    city = input("Enter your preferred city (optional): ").strip().lower()

    return {
        "cuisine": cuisine,
        "max_price": max_price_range,
        "min_rating": min_rating,
        "online_delivery": online_delivery,
        "city": city.lower().strip() if city.strip() else "any",
        "currency": currency
    }

user_preferences = get_user_preferences(df)

for key, value in user_preferences.items():
    print(f"{key.capitalize()}: {value}")

 Please enter your restaurant preferences:


 Available Currencies:
BOTSWANA PULA(P), BRAZILIAN REAL(R$), DOLLAR($), EMIRATI DIRAM(AED), INDIAN RUPEES(RS.), INDONESIAN RUPIAH(IDR), NEWZEALAND($), POUNDS(��), QATARI RIAL(QR), RAND(R), SRI LANKAN RUPEE(LKR), TURKISH LIRA(TL)
Cuisine: japanese
Max_price: 3
Min_rating: 4.0
Online_delivery: no
City: any
Currency: INDIAN RUPEES(RS.)


**STEP-3 :** Implement a content-based filtering
approach where users are recommended
restaurants similar to their preferred criteria.

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def filter_restaurants(df, preferences):
    filtered = df.copy()

    if preferences['cuisine']:
        filtered = filtered[filtered['Cuisines'].str.contains(preferences['cuisine'], case=False, na=False)]

    if preferences['max_price']:
        filtered = filtered[filtered['Price range'] <= preferences['max_price']]

    if preferences['min_rating']:
        filtered = filtered[filtered['Aggregate rating'] >= preferences['min_rating']]

    if 'Has Online delivery' in filtered.columns:
        filtered['Has Online delivery'] = filtered['Has Online delivery'].astype(int)
        if preferences['online_delivery'] == 'yes':
            filtered = filtered[filtered['Has Online delivery'] == 1]
        elif preferences['online_delivery'] == 'no':
            filtered = filtered[filtered['Has Online delivery'] == 0]

    if preferences['city'] != "any":
        filtered = filtered[filtered['City'].str.contains(preferences['city'], case=False, na=False)]

    if preferences['currency']:
        filtered = filtered[filtered['Currency'].str.lower() == preferences['currency'].lower()]

    return filtered

def rank_restaurants(filtered_df, preferences):
    if filtered_df.empty:
        return pd.DataFrame()

    cols_to_combine = ['Cuisines', 'City', 'Currency', 'Rating text']
    for col in cols_to_combine:
        if col in filtered_df.columns:
            filtered_df[col] = filtered_df[col].astype(str).fillna('')

    filtered_df['combined_features'] = filtered_df[cols_to_combine].agg(' '.join, axis=1)

    tfidf = TfidfVectorizer(stop_words='english')
    valid_features_df = filtered_df[filtered_df['combined_features'].str.strip() != ''].copy()

    if valid_features_df.empty:
        return pd.DataFrame()

    tfidf_matrix = tfidf.fit_transform(valid_features_df['combined_features'])
    user_query = f"{preferences['cuisine']} {preferences['city']} {preferences['currency']} {preferences['min_rating']}"
    user_vector = tfidf.transform([user_query])

    similarity_scores = cosine_similarity(user_vector, tfidf_matrix).flatten()
    valid_features_df['similarity'] = similarity_scores

    filtered_df = filtered_df.merge(valid_features_df[['similarity']], left_index=True, right_index=True, how='left')
    filtered_df['similarity'] = filtered_df['similarity'].fillna(0)

    ranked = filtered_df.sort_values(by='similarity', ascending=False).reset_index(drop=True)
    ranked['similarity'] = ranked['similarity'].round(3)

    return ranked[['Restaurant Name', 'Cuisines', 'City', 'Aggregate rating', 'Price range',
                   'Currency', 'Has Online delivery', 'similarity']].head(10)

try:
    filtered_results = filter_restaurants(df, user_preferences)
    final_recommendations = rank_restaurants(filtered_results, user_preferences)

    if not final_recommendations.empty:
        display(final_recommendations)
    else:
        print("No matching restaurants found.")
except NameError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

,Restaurant Name,Cuisines,City,Aggregate rating,Price range,Currency,Has Online delivery,similarity
0,Jonathan's Kitchen - Holiday Inn Express & Suites,"north indian, japanese, italian",hyderabad,4.3,3,INDIAN RUPEES(RS.),0,0.530
1,The Woking Mama,"chinese, thai, japanese, asian",guwahati,4.4,3,INDIAN RUPEES(RS.),0,0.512
2,Yum Yum Cha,"chinese, japanese, sushi",gurgaon,4.5,3,INDIAN RUPEES(RS.),0,0.508
3,Boa Village,"chinese, japanese, korean, asian",new delhi,4.0,3,INDIAN RUPEES(RS.),0,0.417
4,Kylin Experience,"japanese, chinese, thai, malaysian, burmese, a...",chandigarh,4.1,3,INDIAN RUPEES(RS.),0,0.410
5,Mekong - Hotel GreenPark,"chinese, thai, burmese, vietnamese, tibetan, j...",vizag,4.4,3,INDIAN RUPEES(RS.),0,0.389


**STEP-4 :** Test the recommendation system by
providing sample user preferences and
evaluating the quality of recommendations.

In [16]:
print("\n Let's test the restaurant recommendation system!")

user_preferences = get_user_preferences(df)
filtered_df = filter_restaurants(df, user_preferences)
recommended_df = rank_restaurants(filtered_df, user_preferences)

if not recommended_df.empty:
    print("\n Recommended Restaurants for You:\n")
    for idx, row in recommended_df.iterrows():
        print(f"{idx+1}. {row['Restaurant Name']}")
        print(f"Cuisine: {row['Cuisines']}")
        print(f"City: {row['City'].title()}")
        print(f"Rating: {row['Aggregate rating']}")
        print(f"Price Range: {row['Price range']}")
        print(f"Currency: {row['Currency']}")
        print(f"Online Delivery: {'Yes' if row['Has Online delivery'] == 1 else 'No'}")
        print("-" * 40)
else:
    print("\n No restaurants found matching your preferences. Try changing your filters.")


 Let's test the restaurant recommendation system!
 Please enter your restaurant preferences:


 Available Currencies:
BOTSWANA PULA(P), BRAZILIAN REAL(R$), DOLLAR($), EMIRATI DIRAM(AED), INDIAN RUPEES(RS.), INDONESIAN RUPIAH(IDR), NEWZEALAND($), POUNDS(��), QATARI RIAL(QR), RAND(R), SRI LANKAN RUPEE(LKR), TURKISH LIRA(TL)

 Recommended Restaurants for You:

1. Jonathan's Kitchen - Holiday Inn Express & Suites
Cuisine: north indian, japanese, italian
City: Hyderabad
Rating: 4.3
Price Range: 3
Currency: INDIAN RUPEES(RS.)
Online Delivery: No
----------------------------------------
2. The Woking Mama
Cuisine: chinese, thai, japanese, asian
City: Guwahati
Rating: 4.4
Price Range: 3
Currency: INDIAN RUPEES(RS.)
Online Delivery: No
----------------------------------------
3. Yum Yum Cha
Cuisine: chinese, japanese, sushi
City: Gurgaon
Rating: 4.5
Price Range: 3
Currency: INDIAN RUPEES(RS.)
Online Delivery: No
----------------------------------------
4. Boa Village
Cuisine: chinese, japanese